# Postgres Demo notebook

At this stage the coco environment is configured & started.

We have also used a modified deployment, that uses a modified container which includes the Egeria Postgres connector.

We now need to configure the integration server & connector, and point to Postgres from where we will import metadata

In [ ]:
# 
import requests

adminUserId     = "garygeeke"

# // ADD A NEW METADATA SERVER //


mdrServerName = 'dbisrv'

# We already have an integration server - monitorDev01 -- but we'll use a new one for now
intServerName   = "intserv"

adminPlatformURL = devPlatformURL
mdrServerPlatform = devPlatformURL

integrationServiceURLMarker="database-integrator"

eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')

jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

# Pick up which repo type to use from environment if set, otherwise default to inmemory 
metadataRepositoryType   = os.environ.get('repositoryType', inMemoryRepositoryOption)


adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, integrationServiceURLMarker, {})
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)

deployServerToPlatform(adminPlatformURL, adminUserId, mdrServerName, devPlatformURL)

activateServerOnPlatform(mdrServerName, "dev platform", devPlatformURL)

intServerPlatform=devPlatformURL
pgConnectorUser="pguser"
pgConnectorName="postgresql"
pgConnectorClass="org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider"


pgUser="jonesn"
pgPass="science"
pgURL="jdbc:postgresql://158.176.183.28:5432/employees"

configurePlatformURL(adminPlatformURL, adminUserId, intServerName, intServerPlatform)
configureDefaultAuditLog(adminPlatformURL, adminUserId, intServerName)


integrationServiceOptions="{}"
connectorConfigs = [
         {
        "class": "IntegrationConnectorConfig",
        "connectorName": "postgresql",
        "connectorUserId": pgConnectorUser,
        "connection":
        {
          "class": "Connection",
          "userId": pgUser,
          "clearPassword": pgPass,
          "connectorType":
          {
            "class": "ConnectorType",
            "connectorProviderClassName": pgConnectorClass
          },
          "endpoint":
          {
            "class": "Endpoint",
            "address": pgURL"
          },
          "recognizedConfigurationProperties": [
            "url",
            "ssl"
          ],
          "configurationProperties":
          {
            "url": pgURL,
            "ssl": "false"
          }
        },
        "metadataSourceQualifiedName": pgConnectorName,
        "refreshTimeInterval": "3456",
        "usesBlockingCalls": "false",
        "permittedSynchronization": "FROM_THIRD_PARTY"
      }
    ]

configureIntegrationService(adminPlatformURL, adminUserId, daemonServerName, mdrServerName, 
                                mdrServerPlatform, integrationServiceURLMarker, integrationServiceOptions, 
                                connectorConfigs):

deployServerToPlatform(adminPlatformURL, adminUserId, intServerName, devPlatformURL)

activateServerOnPlatform(intServerName, "dev platform", devPlatformURL)